In [ ]:
import numpy as np
import pandas as pd
import json
import csv
import os
import logging
logging.basicConfig()
logging.root.setLevel(logging.INFO)
logger = logging.getLogger()
log = logger.info

In [ ]:
required = [543 , 678, 725, 790, 881, 5278, 8345, 4788, 8345, 9588]

In [ ]:
class portfolio_tracker():
    def __init__(self):
        self.__events = pd.DataFrame(json.load(open("input_data/events.json")))
        self.__fx_details = pd.read_csv("input_data/initial_fx.csv", skipinitialspace=True).set_index("Currency")
        self.__cash_details = pd.read_csv("input_data/initial_cash.csv", skipinitialspace=True).set_index("Desk")
        self.__bond_details = pd.read_csv("input_data/bond_details.csv", skipinitialspace=True).set_index("BondID")
        self.__bond_details["Price"] = np.nan
        self.__meta = pd.DataFrame(columns=['AftEvent', 'Desk', 'Trader', 'Book', 'BondID', 'Price', 'Currency', 'FX', 'Positions', 'NV'])
        self.__exclusions = pd.DataFrame(columns=['EventID', 'Desk', 'Trader', 'Book', 'BuySell', 'Quantity', 'BondID', 'Price', 'ExclusionType'])

    # ----- Getter -----

    def get_events(self):
        return self.__events
    def get_exclusions(self):
        return self.__exclusions
    def get_meta(self):
        return self.__meta
    def get_cash_details(self):
        return self.__cash_details

    # ----- Utilities -----

    def bond_nv(self, bond_quantity, price, fx_rate):
        return bond_quantity*price/fx_rate
    def cash_adjuster(self, desk, value, buysell):
        cash_0 = self.__cash_details.loc[desk, 'Cash']
        cash = (cash_0 - value) if (buysell == 'buy') else (cash_0 + value)
        self.__cash_details.loc[desk, 'Cash'] = round(cash, 2)
    def exclusions_handler(self, type, trade_event, price=0):
        new = {}
        new['EventID'] = trade_event.EventID
        new['Desk'] = trade_event.Desk
        new['Trader'] = trade_event.Trader
        new['Book'] = trade_event.Book
        new['BuySell'] = trade_event.BuySell
        new['Quantity'] = trade_event.Quantity
        new['BondID'] = trade_event.BondID
        new['Price'] = price
        new['ExclusionType'] = type
        self.__exclusions = self.__exclusions.append(new, ignore_index=True)

    # ----- Event Handlers -----

    def price_event_handler(self, price_event):
        # Destructure
        event_id, bond_id, price = price_event[[
            "EventID", "BondID", "MarketPrice"]]

        # Update __bond_details
        self.__bond_details.loc[bond_id, 'Price'] = price

        # Update desk_details
        curr_event = pd.DataFrame(columns=self.__meta.columns)
        for index, row in self.__meta.iterrows():
            # Affected records
            if (row.BondID == bond_id):
                value = self.bond_nv(row.Positions, price, row.FX)
                new_row = row.copy()
                new_row.AftEvent = event_id
                new_row.NV = round(value, 2)
                new_row.Price = price
                curr_event = curr_event.append(new_row, ignore_index=True)
            # Unaffected records
            else:
                new_row = row.copy()
                new_row.AftEvent = event_id
                curr_event = curr_event.append(new_row, ignore_index=True)

        self.__meta = curr_event
    def trade_event_handler(self, trade_event):
        # Destructure
        event_id, desk, trader, book, buysell, quantity, bond_id = trade_event[[
            "EventID", "Desk", "Trader", "Book", "BuySell", "Quantity", "BondID"]]

        # Calculate value
        currency, price = self.__bond_details.loc[bond_id, [
            'Currency', 'Price']]
        rate = self.__fx_details.loc[currency, 'Rate']
        value = self.bond_nv(quantity, price, rate)

        # Check for exclusions
        # Check for NO_MARKET_PRICE
        selected_bond = self.__bond_details.loc[bond_id]
        if (pd.isna(selected_bond.Price)):
            self.exclusions_handler("NO_MARKET_PRICE", trade_event, price)
            return
        # Check for QUANTITY_OVERLIMIT
        selected_book = self.__meta.loc[(self.__meta['Book'] == book) & (
            self.__meta['BondID'] == bond_id)]
        if (buysell == "sell"):
            if (selected_book.empty):
                self.exclusions_handler(
                    "QUANTITY_OVERLIMIT", trade_event, price)
                return
            elif ((selected_book.Positions < quantity).any()):
                self.exclusions_handler(
                    "QUANTITY_OVERLIMIT", trade_event, price)
                return
        # Check for CASH_OVERLIMIT
        selected_desk = self.__cash_details.loc[desk]
        if (buysell == "buy" and selected_desk.Cash < value):
            self.exclusions_handler("CASH_OVERLIMIT", trade_event, price)
            return

        # Update __cash_details
        self.cash_adjuster(desk, value, buysell)

        # Calculate position
        pos_0 = self.__meta.loc[(self.__meta['Book'] == book) & (
            self.__meta['BondID'] == bond_id), 'Positions']
        pos_0 = pos_0.iloc[-1] if (len(pos_0) != 0) else (0)
        pos = (pos_0 + quantity) if (buysell == 'buy') else (pos_0 - quantity)

        # Update self.__meta
        curr_event = pd.DataFrame(columns=self.__meta.columns)
        found = False
        for index, row in self.__meta.iterrows():
            # Affected records
            if (row.Book == book and row.BondID == bond_id):
                value = self.bond_nv(row.Positions, row.Price, row.FX)
                new_row = row.copy()
                new_row.AftEvent = event_id
                new_row.Positions = pos
                new_row.NV = round(value, 2)
                curr_event = curr_event.append(new_row, ignore_index=True)
                found = True
            # Unaffected records
            else:
                new_row = row.copy()
                new_row.AftEvent = event_id
                curr_event = curr_event.append(new_row, ignore_index=True)

        if (found == False):
            new_row = {'AftEvent': event_id, 'Desk': desk, 'Trader': trader, 'Book': book,
                       'BondID': bond_id, 'Price': price,  'Currency': currency, 'FX': rate,
                       'Positions': pos, 'NV': round(value, 2)}
            curr_event = curr_event.append(new_row, ignore_index=True)

        self.__meta = curr_event
    def fx_event_handler(self, fx_event):
        # Destructure
        event_id, ccy, rate = fx_event[["EventID", "ccy", "rate"]]

        # Update __fx_details
        self.__fx_details.loc[ccy] = rate

        # Update self.__meta
        curr_event = pd.DataFrame(columns=self.__meta.columns)
        for index, row in self.__meta.iterrows():
            # Affected records
            if (row.Currency == ccy):
                value = self.bond_nv(row.Positions, row.Price, rate)
                new_row = row.copy()
                new_row.AftEvent = event_id
                new_row.NV = round(value, 2)
                new_row.FX = rate
                curr_event = curr_event.append(new_row, ignore_index=True)
            # Unaffected records
            else:
                new_row = row.copy()
                new_row.AftEvent = event_id
                curr_event = curr_event.append(new_row, ignore_index=True)

        self.__meta = curr_event

    # ----- Report Generators -----

    def gen_bond_report(self, n):
        last_trade_eventID = self.__meta.iloc[-1]['AftEvent'] if (
            not self.__meta.empty) else (-1)
        t = self.__meta.loc[self.__meta['AftEvent']
                            == last_trade_eventID].copy()
        if (not t.empty):
            t['sort1'] = t['Trader'].str.extract('(\d+)')
            t['sort2'] = t['Book'].str.extract('(\d+)')
            t['sort3'] = t['BondID'].str.extract('(\d+)')
            t = t.sort_values(by=['Desk', 'sort1', 'sort2', 'sort3'])
            t = t.drop(['AftEvent', 'Price', 'Currency', 'FX',
                        'sort1', 'sort2', 'sort3'], axis=1)
        else:
            t = t.drop(['AftEvent', 'Price', 'Currency', 'FX'], axis=1)

        t.to_csv(f"bond_level_portfolio_{n}.csv", index=False)
    def gen_currency_report(self, n):
        last_trade_eventID = self.__meta.iloc[-1]['AftEvent'] if (
            not self.__meta.empty) else (-1)
        t = self.__meta.loc[self.__meta['AftEvent']
                            == last_trade_eventID].copy()
        if (not t.empty):
            t = t.groupby(['Desk', 'Currency']).sum().round(2)
            t = t.reset_index()
            t = t.sort_values(['Desk', 'Currency'])
            t = t.drop(['Price', 'FX'], axis=1)
        else:
            t = t.drop(['AftEvent', 'Trader', 'Book',
                        'BondID', 'Price', 'FX'], axis=1)

        t.to_csv(f"currency_level_portfolio_{n}.csv", index=False)
    def gen_position_report(self, n):
        last_trade_eventID = self.__meta.iloc[-1]['AftEvent'] if (
            not self.__meta.empty) else (-1)
        t = self.__meta.loc[self.__meta['AftEvent']
                            == last_trade_eventID].copy()
        if (not t.empty):
            t = t.groupby(['Desk', 'Trader', 'Book']).sum().round(2)
            t = t.reset_index()
            t = t.sort_values(['Desk', 'Trader', 'Book'])
            t = t.drop(['Price', 'FX'], axis=1)
        else:
            t = t.drop(
                ['AftEvent', 'BondID', 'Currency', 'Price', 'FX'], axis=1)

        t.to_csv(f"position_level_portfolio_{n}.csv", index=False)
    def gen_exclusions_report(self, n):
        exclusions_export = self.__exclusions.fillna('')
        exclusions_export.to_csv(f"exclusions_{n}.csv", index=False)
    def gen_cash_report(self, n):
        self.__cash_details.sort_index(inplace=True)
        self.__cash_details.to_csv(f"cash_level_portfolio_{n}.csv")
    def report_generator(self, event_id):
        # Create directory: ./output
        dirName = 'output'
        if not os.path.exists(dirName):
            os.mkdir(dirName)
            log(f"Directory {dirName} created")
        else:
            log(f"Directory {dirName} already exists")

        # Enter directory: ./output
        rootDir = './output'
        os.chdir(rootDir)

        # Create folder for event: ./test/output_i
        subDir = f"output_{event_id}"
        if not os.path.exists(subDir):
            os.mkdir(subDir)
            log(f"Directory {subDir} created")
        else:
            log(f"Directory {subDir} already exists")

        # Enter directory: ./output/output_
        os.chdir(subDir)

        # Create CSV files
        self.gen_exclusions_report(event_id)
        self.gen_cash_report(event_id)
        self.gen_bond_report(event_id)
        self.gen_currency_report(event_id)
        self.gen_position_report(event_id)

        # Go one directory up: ./tests
        os.chdir(os.path.dirname(os.getcwd()))

        # Go one directory up: ./
        os.chdir(os.path.dirname(os.getcwd()))

    # ----- Portfolio Engine -----

    def portfolio_engine(self):
        for index, row in self.__events.iterrows():
            log(f">>> Doing event {index+1}")

            if (row.EventType == "PriceEvent"):
                # log(f">>> Doing event {index+1} (PriceEvent)")
                self.price_event_handler(row)
            elif (row.EventType == "TradeEvent"):
                # log(f">>> Doing event {index+1} (TradeEvent)")
                self.trade_event_handler(row)
            elif (row.EventType == "FXEvent"):
                # log(f">>> Doing event {index+1} (FXEvent)")
                self.fx_event_handler(row)

            if (index+1 in required):
                log(f">>> Generate event report {index+1}")
                self.report_generator(index+1)


In [ ]:
logger.disabled = False
PortfolioTracker = portfolio_tracker()
PortfolioTracker.portfolio_engine()
